In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%reload_ext autoreload

In [3]:
%alias nb_convert ~/bin/develtools/nbconvert createFolders.ipynb

In [4]:
# %nb_convert

In [5]:
import constants
import class_constants
# import & config logging first to prevent any sub modules from creating the root logger
import logging
from logging import handlers
from logging import config
logging.config.fileConfig(constants.logging_config, defaults={'logfile': constants.log_file} )



In [154]:
import csv
import sys
from pathlib import Path
import subprocess
import time
import ArgConfigParse
import os
import glob

In [204]:
def glob_list(start, max_depth=0, min_depth=0, pattern="*", func=os.path.isfile):
    output = []
    for depth in range(min_depth, max_depth):
        path = os.path.join(start, *("*" * depth), pattern)
        output.extend(filter(func, glob.iglob(path)))
    return output

In [235]:
d = {'a': 1, 'b': "cow"}

In [244]:
[k for k in d.keys()]

['a', 'b']

In [263]:
children('/Users/aciuffo/', max_depth=1, child_type='dir')

['/Users/aciuffo/Music',
 '/Users/aciuffo/Dotfiles',
 '/Users/aciuffo/Google Drive File Stream',
 '/Users/aciuffo/bin',
 '/Users/aciuffo/Pictures',
 '/Users/aciuffo/Desktop',
 '/Users/aciuffo/Library',
 '/Users/aciuffo/Public',
 '/Users/aciuffo/Movies',
 '/Users/aciuffo/Applications',
 '/Users/aciuffo/Documents',
 '/Users/aciuffo/Downloads']

In [211]:
glob_list('/Users/aciuffo/Documents/src/portfolioCreator/', max_depth=2, func=os.path.exists)

['/Users/aciuffo/Documents/src/portfolioCreator/student_dirs.txt',
 '/Users/aciuffo/Documents/src/portfolioCreator/logging_cfg.ini',
 '/Users/aciuffo/Documents/src/portfolioCreator/student.export.tsv.text',
 '/Users/aciuffo/Documents/src/portfolioCreator/Untitled.ipynb',
 '/Users/aciuffo/Documents/src/portfolioCreator/xattr.ipynb',
 '/Users/aciuffo/Documents/src/portfolioCreator/portfolioCreator.ipynb',
 '/Users/aciuffo/Documents/src/portfolioCreator/humanfriendly.ipynb',
 '/Users/aciuffo/Documents/src/portfolioCreator/create student folders.ipynb',
 '/Users/aciuffo/Documents/src/portfolioCreator/class_constants.py',
 '/Users/aciuffo/Documents/src/portfolioCreator/constants.py',
 '/Users/aciuffo/Documents/src/portfolioCreator/createFolders.ipynb',
 '/Users/aciuffo/Documents/src/portfolioCreator/__pycache__',
 '/Users/aciuffo/Documents/src/portfolioCreator/consolemenu.ipynb',
 '/Users/aciuffo/Documents/src/portfolioCreator/README.md',
 '/Users/aciuffo/Documents/src/portfolioCreator/Pipf

In [186]:
def glob_list(start, max_depth=0, min_depth=0):
    # start out at least `min_depth` levels deep
    current_dir = os.path.join(start, *"*" * min_depth)
    for depth in range(min_depth, max_depth + 1):
        # go one level deeper
        current_dir = os.path.join(current_dir, "*")
        # print(current_dir)
        yield from filter(os.path.isfile, glob.iglob(current_dir))


if __name__ == "__main__":
    for file_name in glob_list(os.path.expanduser("~/Documents"), max_depth=1, min_depth=0):
        print(file_name)

In [175]:
q = Path('/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)')

In [178]:
depth = 2
for each in q.glob('**/*'):
    base_len = len(str(q).split('/'))
    if each.is_dir():
        if len(str(each).split('/')) - base_len <= depth:
            print(each)

/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022
/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2023
/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022/Harvey, Emma3 - 505552
/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022/Harvey, Emma2 - 505552
/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022/Harvey, Emma1 - 505552
/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022/Fordney, Joseph - 505567
/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-2022/Jaber, Fahad - 505564
/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/C

In [7]:
def csv_to_list(file):
    '''read csv file `file` into a list
    
    Guess the CSV dialect (e.g. tsv, csv, etc.)
    
    Returns `list`'''
    logging.debug(f'reading {file} to list')
    csvFile = Path(file).expanduser().absolute()
    file_csv = []
    # try to figure out the dialect (csv, tsv, etc.)
    with open(csvFile, 'r') as file:
        dialect = csv.Sniffer().sniff(file.read(1024))
        file.seek(0)
        reader = csv.reader(file, dialect)
        for row in reader:
            file_csv.append(row)

    return file_csv

In [8]:
def map_headers(csv_list, expected_headers=[]):
    '''map row 0 of a csv as formatted as a list to a dictionary of expected header values'''
    missing_headers = []
    header_map = {}
    
    csvHeader = csv_list[0]
    logging.debug('mapping headers')
    logging.debug('checking for missing headers')
    for each in expected_headers:
        if each not in csvHeader:
            missing_headers.append(each)
            
    if len(missing_headers) > 0:
        logging.warning(f'missing expected headers: {missing_headers}')
    for index, value in enumerate(csvHeader):
        if value in expected_headers:
            header_map[value] = index
        
    logging.debug('completed mapping')
    return(header_map, missing_headers)

In [9]:
def do_exit(e='unknown error in unknown module: BSoD!', exit_status=99, testing=False):

        
    print('\n'*4)
    if exit_status == 1:
        logging.warning(f'exited before completion with exit code {exit_status}')
        logging.warning(e)  
    elif exit_status > 1:
        logging.error(f'fatal error:\n\t{e}')
    print(e)
    sys.exit(exit_status)
#     if not testing:
#         try:
#             sys.exit(exit_status)
#         except SystemExit:
#             pass

In [343]:
class gd_path():
    def __init__(self, path=None):
        '''google drive path class for files and directories
        
        Attributes:
            path(`str`): path to google drive filestream object'''
        
        self.confirmed = False
        self._file_base = class_constants.file_base
        self._dir_base = class_constants.dir_base
        self.is_file = False
        self.exists = False
        self.path = path

    def __repr__(self):
        return f'gd_path({self.path})'
    
    def __str__(self):
        return f'{self.path}'
    
    @property
    def path(self):
        '''full local path to google dirve filestream object
        
        Args:
            path(`str` or `Path`): /path/to/object
            
        Sets Attributes:
            self.path: path to object
            self.root: same as path for directories, parent directory for files
            self.is_file: true for files and file-like objects, false for directories'''
        return self._path
    
    @path.setter
    def path(self, path):
        if not path:
            self._path = None    
        else:
            self._path = Path(path)
            if self._path.is_dir() and self._path.exists():
                self.root = self._path
                self.is_file = False
                self.exists = True
            if self.path.is_file() and self._path.exists():
                self.root = self._path.parent
                self.is_file = True
                self.exists = True
            
            if not self._path.exists():
                self.is_file = False
                self.root = self._path.parent
                self.exists = False

    @property
    def file_id(self):
        '''unique file id for each object (directories or file)
        
        Args:
            path(`str` or `Path`): path to object; defaults to self.path
        
        Returns:
            `list` of `str` containing the file id'''
        try:
            file_id = self.get_xattr('user.drive.id')
        except FileNotFoundError as e:
            logging.info(f'\'{self.path}\' does not appear to exist; cannot get attributes')
            file_id = None
        return file_id                

    @property
    def webview_link(self):
        '''full webview link to object in google drive'''
        self._webview_link = None
        
        file_id = None
        self._webview_link = None
        
        if self.exists:
            try:
                file_id = self.file_id
            except FileNotFoundError:
                file_id = None


            if len(file_id) < 1:
                file_id = None
            else:
                file_id = file_id[0]

            if not self.is_file and file_id:
                self._webview_link = f'{self._dir_base}{file_id}'

            if self.is_file and file_id:
                self._webview_link = f'{self._file_base}{file_id}'
            
        return self._webview_link
            
    
#     def get_children(self):
#         if (not self.exists) or (self.is_file):
#             self.children = []
#             return self.children
        
#         if self.path:
#             pass
#         else:
#             self.children = []
#             return self.children

    def get_xattr(self, attribute, path=None):
        '''get the extended attributes of a file or directory
        Args:
            attribute('`str`'): attribute key to access

        Returns:
            `list` - attribute or key: attribute pairs
            
        Raises:
            FileNotFoundError - file or directory does not exist
            ChildProcessError - xattr utility exits with non-zero code 
                This is common for files that have no extended attributes or do not
                have the requested attribute'''
        if not path:
            path = self.path
        else:
            path = Path(path).absolute()
            
        attributes = []
        if not self.path.exists():
            raise FileNotFoundError(self.path)

        p = subprocess.Popen(f'xattr -p  {attribute} "{path.resolve()}"', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in p.stdout.readlines():
            attributes.append(line.decode("utf-8").strip())
    #         attributes = attributes + line.decode("utf-8").strip()
        retval = p.wait()
        if retval != 0:
            raise ChildProcessError(f'"{path}" is likely not a google filestream object: xattr exited with code {retval}')
        return attributes    
    
    
    def children(self, path=None, max_depth=1, pattern="*", child_type='all'):
        '''return list of all children matching a pattern of `child_type`

        Args:
            path(`str`): path to recurse
            pattern(`str`): pattern to match (default "*")
            child_type(`str`): "all" files and dirs (default); "file" file only; "dir" directory only

        Returns:
            `list`

        based on: https://codereview.stackexchange.com/questions/161989/recursively-listing-files-in-python/162322'''
        if not path:
            path = self.path
        output = []
        known_child_types = {'all': os.path.exists, 
                             'file': os.path.isfile, 
                             'dir': os.path.isdir}
        if not child_type in known_child_types.keys():
            raise ValueError(f'"{child_type}" not a known type: {[k for k in known_child_types.keys()]}')
        else:
            func = known_child_types[child_type]

        for depth in range(0, max_depth):
            path = os.path.join(path, *("*" * depth), pattern)
            output.extend(filter(func, glob.iglob(path)))
        return output
    
    def mkdir(self, path=None, parents=False, exist_ok=False, **kwargs):
        '''create a directory using pathlib.Path().mkdir()
        
        Args:
            path(`str` or `Path`): path to create
            parents(`bool`): create parent directories - default false
            exists_ok(`bool`): do not raise error if directory exists
            kwargs: kwargs for pathlib.Path().mkdir()
            
        Returns:
            `list` containing file_id'''
        if not path:
            path = self.path
            logging.debug(f'using self.path: {path}')
        else:
            logging.debug(f'using supplied path: {path}')
            
        if path.is_file():
            raise TypeError(f'{path} is a file')
            
        path = Path(path)
            
        path.mkdir(parents=parents, exist_ok=exist_ok, **kwargs)
        if self.confirm(path):
            file_id = self.get_xattr('user.drive.id', path)
        return self.file_id        
    
    def confirm(self, path=None):
        '''confirm that an object has been synced over filesgtream
        
        Args:
            path(`str` or `Path`): path to object; default is self.path
        
        Returns:
            `list` of `str` containing the file id
            
        Attributes Set:
            self.confirmed: True when object has been sent'''
        
        if not path:
            path = self.path
        file_id = self.file_id
        
        if file_id:
            if 'local-' in file_id[0]:
                self.confirmed = False
                file_id = None
            else:
                self.confirmed = True
        return file_id
    
    @classmethod
    def mkchild(cls, path, **kwargs):
        '''create child directory and return a gd_path object'''
        child = cls(path=path)
        child.mkdir(**kwargs)
        child.confirm()
        return child

In [ ]:
# class xgd_path():
#     def __init__(self, path=None):
#         '''google drive path class
        
#         Attributes:
#             path(`str`): path to google drive drive object'''
#         self.confirmed = False
#         self.path = path
#         self._file_base = 'https://drive.google.com/file/d/'
#         self._dir_base = 'https://drive.google.com/drive/folders/'
#         self.is_file = False   
    
    
#     def __repr__(self):
#         return f'gd_path({self.path})'
    
#     def __str__(self):
#         return f'{self.path}'
    
#     @property
#     def path(self):
#         return self._path
    
#     @path.setter
#     def path(self, path):
#         '''full path to object
        
#         Args:
#             path(`str` or `Path`): /path/to/object
            
#         Sets Attributes:
#             self.path: path to object
#             self.root: same as path for directories, parent directory for files
#             self.is_file: true for files and file-like objects, false for directories'''
#         if not path:
#             self._path = None
#         else:
#             self._path = Path(path)
#             if self._path.is_dir() and self._path.exists():
#                 self.root = self._path
#                 self.is_file = False
#             if self.path.is_file() and self._path.exists():
#                 self.root = self._path.parent
#                 self.is_file = True
            
#             if not self._path.exists():
#                 self.is_file = False
#                 self.root = self._path.parent

#     @property
#     def webview_link(self):
#         '''full webview link to object in google drive'''
#         self._webview_link = None
#         try:
#             item_id = self.get_xattr('user.drive.id')
#         except FileNotFoundError as e:
#             logging.debug(f'{e}')
#             return None
#         except ChildProcessError as e:
#             logging.debug(f'{e}')
#             return None

#         if len(item_id) < 1:
#             return None
#         else:
#             item_id = item_id[0]
        
        
#         if not self.is_file:
#             self._webview_link = f'{self._dir_base}{item_id}'
#         if self.is_file:
#             self._webview_link = f'{self._file_base}{item_id}'
#         return self._webview_link
            
#     def check_parent(self, expected):
#         '''checks if the parent matches the expected parent'''
#         if self.root.parents[0].name == expected:
#             return True
#         else:
#             return False
        
#     def get_xattr(self, attribute, file=None):
#         '''get the extended attributes of a file or directory
#         Args:
#             file(`str` or Path): path to file
#             attribute('`str`'): attribute key to access

#         Returns:
#             `list` - attribute or key: attribute pairs

#         Raises:
#             FileNotFoundError - file or directory does not exist
#             ChildProcessError - xattr utility exits with non-zero code 
#                 This is common for files that have no extended attributes or do not
#                 have the requested attribute'''
#         if not file:
#             file = self.path
#         else:
#             file = Path(file).absolute()
            
#         attributes = []
#         if not file.exists():
#             raise FileNotFoundError(file)

#         p = subprocess.Popen(f'xattr -p  {attribute} "{file.resolve()}"', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
#         for line in p.stdout.readlines():
#             attributes.append(line.decode("utf-8").strip())
#     #         attributes = attributes + line.decode("utf-8").strip()
#         retval = p.wait()
#         if retval != 0:
#             raise ChildProcessError(f'xattr exited with value: {retval}')
#         return attributes     

#     @property
#     def file_id(self, path=None):
#         '''unique file id for each object (directories or file)
        
#         Args:
#             path(`str` or `Path`): path to object; defaults to self.path
        
#         Returns:
#             `list` of `str` containing the file id'''
#         if not path:
#             path = self.path
#         try:
#             file_id = self.get_xattr('user.drive.id', path)
#         except FileNotFoundError as e:
#             logging.info(f'\'{path}\' does not appear to exist; cannot get attributes')
#             file_id = None
#         return file_id
    
#     def confirm(self, path=None):
#         '''confirm that a created object has been sent over file stream
        
#         Args:
#             path(`str` or `Path`): path to object; default is self.path
        
#         Returns:
#             `list` of `str` containing the file id
            
#         Attributes Set:
#             self.confirmed: True when object has been sent'''
        
#         if not path:
#             path = self.path
#         file_id = self.file_id
        
#         if file_id:
#             if 'local-' in file_id[0]:
#                 self.confirmed = False
#                 file_id = None
#             else:
#                 self.confirmed = True
#         return file_id
    
#     def mkdir(self, path=None, parents=False, exist_ok=False, kwargs={}):
#         '''create a directory using pathlib.Path().mkdir()
        
#         Args:
#             path(`str` or `Path`): path to create
#             parents(`bool`): create parent directories - default false
#             exists_ok(`bool`): do not raise error if directory exists
#             kwargs: kwargs for pathlib.Path().mkdir()
            
#         Returns:
#             file_id(`list`)'''
#         if not path:
#             path = self.path
#             logging.debug(f'using self.path: {path}')
#         else:
#             logging.debug(f'using supplied path: {path}')
            
#         if path.is_file():
#             raise TypeError(f'{path} is a file')
            
#         path = Path(path)
            
#         path.mkdir(parents=parents, exist_ok=exist_ok, **kwargs)
#         if self.confirm(path):
#             file_id = self.get_xattr('user.drive.id', path)
#         return self.file_id
    
#     def get_children(self, children):
#         '''return children directories of a directory'''
#         pass

In [419]:
class student_path(gd_path):
    def __init__(self, root=None, ClassOf=None, Student_Number=None, LastFirst=None):
        '''student directory in google drive; child class of gd_path:
        
        Args:
        
        Properties:
            class_of(`str`): "ClassOf-YYYY" string representation of projected graduation year
            name(`str`): "Last, First" string representation of student name
            id_number(`int`): student id number
            matches(`dict`):  name and webview link of directories that contain "id_number"
            path_parts(`dict`): path compontents stored as dictionary keys'''
        
        super(student_path, self).__init__(path=root)       
        self.matches = {}
        self.path_parts = {'ClassOf': None, 'Student_Number': None, 'LastFirst': None}
        self.ClassOf = ClassOf
        self.LastFirst = LastFirst
        self.Student_Number = Student_Number
        
    def __repr__(self):
        return f'student_path({self.path})'
        
    def __str__(self):
        return f'{self.path}'
    
    def set_path(self):
        for key in self.path_parts:
            if not self.path_parts[key]:
                return False
        student_dir = f"ClassOf-{self.path_parts['ClassOf']}/{self.path_parts['LastFirst']} - {self.path_parts['Student_Number']}"
        student_dir = f'{str(self.root)}/{student_dir}'
        self.path = student_dir

    
    @property
    def ClassOf(self):
        return se.f.path_parts['ClassOf']
    
    @ClassOf.setter
    def ClassOf(self, ClassOf):
        '''string representation of projected graduation date in format: "ClassOf-YYYY"
        
        Properties Set:
            path_parts(`dict`): dictionary of component parts of path'''
        if not ClassOf:
            self.path_parts['ClassOf'] = None
        else:
            # attempt to coerce strings from cSV file into type int
            ClassOf = int(ClassOf)
            if not isinstance(ClassOf, int):
                raise TypeError('class_of must be of type `int`')
        self.path_parts['ClassOf'] = f'ClassOf-{ClassOf}'
        self.path_parts['ClassOf'] = ClassOf
        self.set_path()
        
        
    @property
    def LastFirst(self):
        return self.path_parts['LastFirst']
    
    @LastFirst.setter
    def LastFirst(self, name):
        '''string representation of "Last, First" names
        
        Properties Set:
            path_parts(`dict`): dictionary of component parts of path'''
        if not name:
            self.path_parts['LastFirst'] = None
        else:
            if not isinstance (name, str):
                raise TypeError('name must be of type `str`')
        self.path_parts['LastFirst'] = name
        self.set_path()
        
    @property
    def Student_Number(self):
        return self.path_parts['Student_Number']
    
    @Student_Number.setter
    def Student_Number(self, number):
        '''integer of student id number
        
        Properties Set:
            path_parts(`dict`): dictionary of component parts of path'''
        if not number:
            self.path_parts['Student_Number'] = None
        else:
            # try to coerce number into type int
            number = int(number)
            if not isinstance (number, int):
                raise TypeError('id_number must be of type `int`')
        self.path_parts['Student_Number'] = number  
        self.set_path()
        

In [ ]:
# class xstudent_path(gd_path):
#     def __init__(self, path=None, class_of=None, id_number=None, name=None):
#         '''student directory in google drive; child class of gd_path:
        
#         Args:
        
#         Properties:
#             class_of(`str`): "ClassOf-YYYY" string representation of projected graduation year
#             name(`str`): "Last, First" string representation of student name
#             id_number(`int`): student id number
#             matches(`dict`):  name and webview link of directories that contain "id_number"
#             path_parts(`dict`): path compontents stored as dictionary keys'''
        
#         super(student_path, self).__init__(path=path)
#         self.matches = {}
#         self.path_parts = {'ClassOf': None, 'id_number': None, 'name': None}
#         self.class_of = class_of
#         self.name = name
#         self.id_number = id_number
    
    
#     def __repr__(self):
#         return f'student_path({self.student_dir_name})'
        
#     def __str__(self):
#         return f'{self.student_dir_name}'
    
#     def get_xattr(self, attribute, file=None):
#         if not file:
#             file = self.student_dir_name
#         return super().get_xattr(attribute, file)
    
#     @property
#     def class_of(self):
#         return self._class_of
    
#     @class_of.setter
#     def class_of(self, class_of):
#         '''string representation of projected graduation date in format: "ClassOf-YYYY"
        
#         Properties Set:
#             path_parts(`dict`): dictionary of component parts of path'''
#         if not class_of:
#             self._class_of = None
#         else:
#             # attempt to coerce strings from cSV file into type int
#             class_of = int(class_of)
#             if not isinstance(class_of, int):
#                 raise TypeError('class_of must be of type `int`')
#         self.path_parts['ClassOf'] = f'ClassOf-{class_of}'
#         self._class_of = class_of
        
#     @property
#     def name(self):
#         return self._name
    
#     @name.setter
#     def name(self, name):
#         '''string representation of "Last, First" names
        
#         Properties Set:
#             path_parts(`dict`): dictionary of component parts of path'''
#         if not name:
#             self._name = None
#         else:
#             if not isinstance (name, str):
#                 raise TypeError('name must be of type `str`')
#         self.path_parts['name'] = name
#         self._name = name
        
#     @property
#     def id_number(self):
#         return self._id_number
    
#     @id_number.setter
#     def id_number(self, number):
#         '''integer of student id number
        
#         Properties Set:
#             path_parts(`dict`): dictionary of component parts of path'''
#         if not number:
#             self._id_number = None
#         else:
#             # try to coerce number into type int
#             number = int(number)
#             if not isinstance (number, int):
#                 raise TypeError('id_number must be of type `int`')
#         self.path_parts['id_number'] = number
#         self._id_number = number

#     @property
#     def student_dir_name(self):
#         '''full absolute path to student directory in format:
#             ClassOf-YYYY/Last, First - NNNNNNN'''
#         d = f"/{self.path_parts['ClassOf']}/{self.path_parts['name']} - {self.path_parts['id_number']}"
#         if self.path:
#             # not sure why this is needed, but any joining of self.root/Path(d) fails
#             d = f'{str(self.path)}/{d}'
#         return Path(d)
    
#     # method for checking for similarly named student folders in this ClassOf folder
#     def check_similar(self):
#         '''check for similarly named directories based on student id number 
#         within the path/ClassOf/ directory
        
#         Properties Set:
#             self.matches(`dict`): dictionary of similar directories
#         Returns:
#             `bool`: True if matching directories found'''
#         similar = False
#         matches = {}
#         for i in self.student_dir_name.parent.glob(f"*{self.path_parts['id_number']}*"):
#             match_id = self.get_xattr('user.drive.id', self.student_dir_name.parent/i)
#             if i.absolute().is_dir():
#                 url = '/'.join((self._dir_base, match_id[0]))
#             else:
#                 url = '/'.join((self._file_base, match_id[0]))
#             matches[str(i)] = url
#         self.matches = matches
#         if matches:
#             similar = True
#         return similar

#     def mkdir(self, path=None, exist_ok=False, parents=True, kwargs={}):
#         '''make a google drive directory using pathlib.Path().mkdir()
        
#         Args:
#             path(`str` or `Path`): defaults to self.student_dir_name
#             exist_ok(`bool`): True - do not raise error if directory exists
#             parents(`bool`): True - create parents if they do not exist
#             kwargs({}): pathlib.Path() kwargs
            
#         Returns:
#             list[str]: google drive object ID string'''
#         if not path:
#             path = self.student_dir_name
#         logging.debug(f'calling super().mkdir(path={path})')
#         val = super().mkdir(path=path, exist_ok=exist_ok, parents=parents, **kwargs)
#         return val
            
        

In [ ]:
def validate_data(csv_list, expected_headers, header_map):
    '''validate list items for proper data types
         naievely attempts to coerce strings from CSV into expected_header types
         returns a tuple of list of rows that were successfully coerced and those
         that could not be coerced
    
    Args:
        csv_list(`list` of `list`): csv as a nested list [['h1', 'h2'], ['item', 'item2']]
        expected_headers(`dict`): {'literal_header': type} {'ClassOf':, int, 'Name', str}
        header_map(`dict`): map of list index for each header {'h1': 0, 'h2': 5, 'hN': i}
        
    Returns:
        (`tuple` of `list`): (valid_rows, invalid_rows)
    '''
    valid = []
    invalid = []

    for row in csv_list[1:]:
        good_row = True
        for k in expected_headers.keys():
            # test for coercable types
            try:
                test = expected_headers[k](row[header_map[k]])
            except ValueError:
#                 do_exit(f'Bad student.export: {k} contained {row[header_map[k]]}\ncannot continue. Please try running the export again.')
                logging.warning(f'bad row: {row}')
                logging.warning(f'column "{k}" contained "{row[header_map[k]]}"--this should be {(expected_headers[k])}')
                invalid.append(row)
                good_row = False
                break
        if  good_row:
            valid.append(row)
        
    return valid, invalid
    

In [ ]:
def adjust_handler(handler=None, new_level=None):
    '''adjust a logging handler
    
    Args:
        handler(`str`): partial string in handler name - if none, returns list of all handlers attached to root
            '*' adjusts all handlers to new_level
        new_level(`str`): DEBUG, INFO, WARNING, ERROR
    
    Returns:
        `list`: list of handlers and levels currently set'''
    if not handler:
        return(logging.getLogger().handlers)
    
    my_handler = None    
    for index, val in enumerate(logging.getLogger().handlers):
        if handler == '*':
            my_handler = logging.getLogger().handlers[index]
        else:
            if handler in str(val):
                my_handler = logging.getLogger().handlers[index]
        if my_handler:
            logging.info(f'setting {str(my_handler)} to {new_level}')
            my_handler.setLevel(new_level)
        else:
            logging.warning(f'handler: "{handler}" not found')
        
    return logging.getLogger().handlers

In [ ]:
def parse_cmdargs():
    '''set known command line arguments, parse sys.argv
    
    Returns:
        `dict`: nested dictionary of command line arguments that matches strcture of .ini file'''
    args = ArgConfigParse.CmdArgs()
    args.add_argument('-s', '--student_export', ignore_none=False, metavar='/path/to/student.export.csv', 
                      type=str, dest='student_export', help='Export from PowerSchool containing: LastFirst, ClassOf, Student_Number')

    args.add_argument('-g', '--google_drive', ignore_none=True, metavar='/Volumes/GoogleDrive/Shared drives/ASH Cum Folders/folder/',
                      type=str, dest='main__drive_path', help='Full path to Google Drive Shared Drive containing cumulative files')

    args.add_argument('-l', '--log_level', ignore_none=True, metavar='ERROR, WARNING, INFO, DEBUG', 
                      type=str, dest='main__log_level', help='Logging level -- Default: WARNING')

    args.parse_args()
    return args.nested_opts_dict                  

In [ ]:
def read_config(files):
    '''parse .ini files 
    
    Args:
        files(`list`): list of `str` containing files in .ini format to parse
    
    Returns:
        `dict`: nested dict of configuration'''
    parser = ArgConfigParse.ConfigFile(config_files=files, ignore_missing=True)
    parser.parse_config()
    
    return parser.config_dict

In [ ]:
def check_drive_path(drive_path=None):
    '''check that path is a valid google drive path and contains the appropriate sentry file
    
    Args:
        drive_path(`str`): path to google drive containg cummulative folders and sentry file
    
    Retruns:
        `tuple` of `bool`, `str`: When true, drive is OK; when false, drive is not valid; str contains errors'''
    # this is super redundant -- checks the following:
    # * is a path
    # * is a google drive path
    # * if sentry file exists
    # this may be a good idea considering how some users have run into many problems with this

    drive_ok = True
    msg = None
    if not drive_path:
        logging.info('no google drive specified')
        drive_ok = False
        msg = 'No Google Drive specified'
        return drive_ok, msg
    else:
        drive_path = Path(drive_path)
    
    if not drive_path.exists():
        logging.warning(f'specified path "{drive_path}" does not exist')
        drive_ok = False
        msg = f'The Google Drive "{drive_path}" does not appear to exist on Google Drive'
        return drive_ok, msg
    else:
        google_drive = gd_path(drive_path)
    
    try:
        google_drive.get_xattr('user.drive.id')
    except ChildProcessError as e:
        logging.warning(f'specified path "{drive_path}" is not a Google Drive path')
        msg = f'The Google Drive "{drive_path}" does not appear to be a valid google Shared Drive'
        drive_ok = False
        return drive_ok, msg

    sentry_file = constants.sentry_file    
    sentry_file_path = drive_path/Path(sentry_file)
    
    if not sentry_file_path.is_file():
        logging.warning(f'sentry file is missing in specified path "{drive_path}"')
        msg = f'''The chosen google shared drive "{drive_path}"
does not appear to be a Cumulative Student Folder. 

The file: "{sentry_file}" is missing. 
If you are sure {drive_path} is correct, 
please contact IT Support and askfor help. 

Please screenshot or copy this entire text below and provide it to IT Support.

###############################################################################
Run the command below from the terminal of the user that submitted this ticket.
This command will create the necessary files for this script. 

Confirm that {drive_path} is the correct
Google Shared Drive for Cumulative Student Folders BEFORE proceeding.
     $ touch {drive_path}/{sentry_file}'''
        drive_ok = False
    
    
    
    return drive_ok, msg

In [ ]:
def create_folders(valid_rows=[], invalid_rows=[], header_map=[], drive_path=None):
    '''create folders in drive_path from valid_rows and header_map; validate the creation of each directory
    
    Args:
        valid_rows(`list` of `list`): validated rows from CSV file to create
        invalid_rows(`list` of `list`): invalid rows from CSV file to skip
        header_map(`dict`): dictionary that maps headers to column in CSV
        drive_path(`Path`): path to create folders
    
    Returns:
        `dict` of `list`: dictionary containing list of student_path objects created, failed, or skipped
    '''
    # try to confirm created files N times before giving up
    confirm_retry = constants.confirm_retry
    #  wait N seconds for first try, N*retry for each subsiquent retry
    base_wait = constants.base_wait    
    
    if len(valid_rows) < 1:
        do_exit('No valid rows were found', 1)
    if len(header_map) < 1:
        do_exit('bad or missing header map', 1)
        
    if not drive_path or not isinstance(drive_path, (Path)):
        do_exit('bad or missing drive_path', 1)

    directories = {'created': [], 'skipped': [], 'invalid': invalid_rows, 
                   'confirmed': [], 'failed': []}
    
    # work through validated rows
    logging.debug('processing valid rows')
    for row in valid_rows:
        name = row[header_map['LastFirst']]
        class_of = row[header_map['ClassOf']]
        id_number = row[header_map['Student_Number']]
        
        s_path = student_path(path=drive_path,
                              name=name, 
                              class_of=class_of, 
                              id_number=id_number)
        
        # check if there already exists a directory with the student number
        logging.debug(f'checking for similarly named folders for {name} - {id_number}')
        if s_path.check_similar():
            # flag those that have multiple entries
            if len(s_path.matches) > 1:
                logging.warning(f'multiple directories exist in {class_of} for student number {id_number}')
                directories['skipped'].append((s_path, f'multiple: {len(s_path.matches)} existing folders found'))
            # flag those that already exist for auditing purposes
            else:
                logging.info(f'skipped {class_of}/{name} - {id_number}: folder exists')
                directories['skipped'].append((s_path, 'exists'))

                
        else:
            # create the directory and try to handle errors as needed
            try:
                s_path.mkdir(parents=True)
            except FileExistsError as e:
                logging.error(f'{s_path.student_dir_name} exists')
                directories['skipped'].append((s_path, 'exists'))
            except OSError as e:
                logging.error(f'Could not create {s_path.student_dir_name}: {e}')
                directories['failed'].append((s_path, 'error: {e}'))
            else:
                directories['created'].append(s_path)

# inject a bad entry to test checks at end
#     directories['created'].append(student_path(path='/Volumes/GoogleDrive/Shared drives/IT Blabla I/spam_eggs_spam',
#                                                name='Eggs, Green', class_of=1000, id_number=123456))
    
    
    # double check that drectories were created and properly synced to google drive
    logging.info('confirming created directories have synced')
    for i in range(0, confirm_retry):
        dirs_to_check = directories['created']
        logging.debug(f'attempt {i} of {confirm_retry}')
        logging.debug(f'{len(dirs_to_check)} directories remain to be confirmed')
        if len(dirs_to_check) > 0:
            wait = i * base_wait
        for each in dirs_to_check:
            logging.info(f'checking: {each}')
            if each.confirm():
                logging.debug(f'confirmed: {each}')
                directories['confirmed'].append(each)
                directories['created'].remove(each)
            else:
                logging.info(f'not confirmed')
            
        # loop over the created directories N times with a longer delay each time
        # check that everything is confirmed uploaded; if it is not after Nth time, 
        # log as 'failed'
        if len(directories['created']) > 0:
            logging.info(f'sleeping for {wait} seconds and checking dirs again')
            time.sleep(wait)
        else:
            logging.info('all created directories confirmed')
            break

    
    # anything left in created is unconfirmed and considered failed
    if len(directories['created']) > 0:
        for each in directories['created']:
            directories['failed'].append((each, 'error: could not confirm folder was created on google drive\n\ttry the same export again later'))
        # zero out created
        directories['created'] = []
        
        
    return directories

In [ ]:
def main():    
    # set the local logger
    logger = logging.getLogger(__name__)
#     logger = logging.LoggerAdapter(logging.getLogger(__name__), {'N': 'bananas'})

    # base configuration fle
    config_file = Path(constants.config_file)
    # user config file (~/.config/app_name/app.ini)
    user_config_path = Path(constants.user_config_path)
    
    # if the user configuration file is missing set to True & create later at end
    update_user_config = not(user_config_path.exists)
    logging.debug(f'update_user_config: {update_user_config}')

    # parse command line and config files - 
    cmd_args_dict = parse_cmdargs()
    cfg_files_dict = read_config([constants.config_file, constants.user_config_path])

    # merge the command line arguments and the config files; cmd line overwrites files
    config = ArgConfigParse.merge_dict(cfg_files_dict, cmd_args_dict)

    # adjust the logging levels if needed
    if config['main']['log_level']:
        ll = config['main']['log_level']
        if ll in (['DEBUG', 'INFO', 'WARNING', 'ERROR']):
            logging.root.setLevel(ll)
            handlers = adjust_handler('*', ll)
            logging.debug(f'adjusted log levels: {handlers}')
        else:
            logging.warning(f'unknown or invalid log_level: {ll}')
    
    # load file constants
    expected_headers = constants.expected_headers
    
    student_dirs = constants.student_dirs
        
    # get csv_file and drive_path from the command line
    try:
        csv_file = Path(config['__cmd_line']['student_export'])
    except TypeError:
        logging.info('No student export file specified on command line')
        csv_file = None
        
    
    # check drive path is a google drive path
    drive_path = Path(config['main']['drive_path'])
    drive_status = check_drive_path(drive_path)
    
    if not drive_status[0]:
        do_exit(sentry_status[1], 1)
        # consider prompting user at this point to enter a valid drive
    
    # read CSV into a list
    if not csv_file:
        do_exit('No student export CSV file specified. Exiting.', 1)
    try:
        csv_list = csv_to_list(csv_file)
    except (FileNotFoundError, OSError, IOError, TypeError) as e:
        logging.error(f'could not read csv file: {csv_file}')
        logging.error(f'{e}')
        do_exit(e, 1)
    
    # map the expecdted headers to the appropriate columns
    header_map, missing_headers = map_headers(csv_list, expected_headers.keys())
    
    # error out if there are any missing headers in the export file
    if len(missing_headers) > 0:
        do_exit(f'{csv_file.name} is missing one or more headers:\n\t{missing_headers}\nprogram cannot continue', 1)
    
    # validate the csv list
    valid_rows, invalid_rows = validate_data(csv_list, expected_headers, header_map)

    # create folders from valid rows and header_map
    directories = create_folders(valid_rows=valid_rows, invalid_rows=invalid_rows, header_map=header_map,
                                drive_path=drive_path)
    
    # handle students with multiple entries
    multiple_entries = [i[0] for i in directories['skipped'] if i[1] =='multiple' ]

    print('created new:')
    for each in directories['confirmed']:
        print(each.webview_link)
        
        
    print('confirmed exist:')
    for each in directories['skipped']:
        print(each[0].webview_link)
            
#     if len()
#         print('Students with multiple portfolio folders:')
#         print('The students below have multiple cumulative folders. This is likely due to a student name change.')
#         print('\nYOU MUST PICK **ONE** FOLDER AND MOVE ALL THE STUDENT DATA INTO THAT ONE FOLDER.\nDELETE THE OTHERS WHEN DONE.\n\nTHIS IS A MAJOR PROBLEM.')

#         for each in multiple:
#             if each.check_similar:
#                 print(f'ClassOf-{each.class_of}: {each.name} - {each.id_number} has {len(each.matches)} folders:')
#                 for folder in each.matches:
#                     print(f'\t{each.matches[folder]}')

    
        
    if update_user_config:
        try:
            logging.info(f'updating user configuration file: {user_config_path}')
            ArgConfigParse.write(config, user_config_path, create=True)
        except Exception as e:
            m = f'Error updating user configuration file: {e}'
            do_exit(m, 1)

            
    # add summary of actions and errors
    # create csv output for adding portfolio links into PS SIS
    return directories
    

    # cleanup
    # handle invalid_rows -- notify user of issues


In [ ]:
def write_output_csv(directories):
    
    output_list = []
    issue_list = []
    
    total = 0
    
    for entry in directories['confirmed']:
        # these are failures
        pass
    
    for entry in directories['skipped']:
        if entry[1] == 'exists':
            output_list.append(entry)
        if 'multiple:' in entry[1]:
            issue_list.append(entry)
            
    for entry in directories['failed']:
        issue_list.append(entry)

    
    total = total + len(issue_list)
    total = total + len(output_list)
        
    if len(output_list) > 0:
        logging.info(f'successfully created: {len(output_list)} of {total} folders')
        print(f'\nFolders were created or already existed for {len(output_list)} students')       
        
    if len(issue_list) > 0:
        logging.info(f'failed to create or skipped: {len(issue_list)} of {total} folders')
        print('\nThe folders below were not created for the following reasons:')
        for each in issue_list:
            print(f'* ClassOf-{each[0].class_of}/{each[0].name} - {each[0].id_number}')
            print(f'\t{each[1]}')           
    

    multiple = [i[0] for i in issue_list if 'multiple:' in i[1]]
    if len(multiple) > 0:
            print('\nStudents with multiple portfolio folders:')
            for each in multiple:
                if each.check_similar:
                    print(f'* {each.name}')
                    for folder in each.matches:
                        print(f'\t{each.matches[folder]}')
            print('''YOU MUST MERGE THESE SO THERE IS ONLY FOLDER FOR EACH STUDENT. THIS IS A MAJOR PROBLEM.

**Steps to resolve the above issue**
\t01. Choose one of the folders above. 
\t02. Open each of the folders and move any student work into the chosen folder.
\t03. Manually delete the folders that are no longer needed.''')
            
        

In [ ]:
write_output_csv(f)

In [ ]:
if __name__ == '__main__':
    f = main()

In [ ]:
f

In [ ]:
each = f['skipped'][0][0]

In [ ]:
each.class_of

In [ ]:
    skipped = len(directories['skipped'])
    failed = len(directories['failed'])
    confirmed = len(directories['confirmed'])
    invalid = len(directories['invalid'])
    multiple = [i[0] for i in skipped if i[1] =='multiple' ]

    print('created new:')
    for each in directories['confirmed']:
        print(each.webview_link)
        
        
    print('confirmed exist:')
    for each in directories['skipped']:
        print(each[0].webview_link)
            
    print('Students with multiple portfolio folders:')
    for each in multiple:
        if each.check_similar:
            print(each.name)
            for folder in each.matches:
                print(each.matches[folder])
        
   

In [ ]:
s = f['skipped']

In [ ]:
mult = [m[0] for m in f['skipped'] if m[1] =='multiple' ]

In [ ]:
mult

In [ ]:
m = s[0]

In [ ]:
e = m[0]

In [ ]:
e.check_similar()

In [ ]:
print(e.name)
for s in e.matches:
    print(e.matches[s])

In [ ]:
sys.argv.append('-g')
sys.argv.append('/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)')

In [ ]:
sys.argv.append('-g')
sys.argv.append('/xVolumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)')

In [ ]:
sys.argv.append('-s')
# sys.argv.append('./student.export.text')
sys.argv.append('./invalid.student.export.text')
# sys.argv.append('./bad.student.export.text')

In [ ]:
sys.argv.append('-l')
sys.argv.append('INFO')

In [ ]:
sys.argv.pop()

In [ ]:
# sys.argv.append('-s')
# sys.argv.append('./student.export.csv.text')
# # f = main()

In [ ]:
sys.argv

In [ ]:
# sys.argv.append('-g')
# sys.argv.append('/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)')